In [1]:
import os
import sys
import numpy as np
import pandas as pd
import copy
from os.path import join, exists

In [2]:
sys.path.append('../../')
from src.utils import split_gen, sampling, data_cleaning, load_models, data_cleaning, transformers_bert_completions, configuration
config = configuration.Config()
np.random.seed(config.SEED)

/Users/nicolewong/opt/anaconda3/envs/child_grammar/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
final_save_path = join(config.prov_csv_dir, 'pvd_utt_glosses_phono_cleaned_inflated_to_next_notebook.pkl')
all_tokens_phono = pd.read_pickle(final_save_path)

## Get the samples and splits for age/all splits

In [4]:
# Do this for each of success and yyy, then merge them together.

all_tokens_phono_valid = data_cleaning.find_transcripts_with_successes_and_yyy(all_tokens_phono)

# Split train/val/test: 25/50/50.

split_attr = 'transcript_id'

phono_train_val_idxs, phono_eval_idxs = split_gen.determine_split_idxs(all_tokens_phono_valid, split_attr, 0.5)

phono_train_val = all_tokens_phono_valid[all_tokens_phono_valid.transcript_id.isin(phono_train_val_idxs)]
phono_train_idxs, phono_val_idxs = split_gen.determine_split_idxs(phono_train_val, split_attr, 0.5)

for phase, idx_set in zip(['train', 'val', 'eval'], [phono_train_idxs, phono_val_idxs, phono_eval_idxs]):

    # It's on transcript_id, not actual idx, so this is OK.
    # all_tokens_phono will receive the val/eval phase marking where it applies.

    this_phase_data, all_tokens_phono = split_gen.assign_and_find_phase_data(phase, split_attr, idx_set, all_tokens_phono)

all_tokens_phono = data_cleaning.augment_target_child_year(all_tokens_phono)

# Below: For debugging only

all_tokens_phono.to_pickle(join(config.prov_dir, 'pvd_all_tokens_phono_for_eval_before_child.pkl')) 


In [5]:
import imp
imp.reload(sampling)
from src.utils import paths
imp.reload(paths)
imp.reload(configuration)
imp.reload(split_gen)
config = configuration.Config()

/var/folders/g7/6fmhhq2x5813j_xlfxrqxln80000gn/T/ipykernel_15154/851981157.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [6]:
used_ages = data_cleaning.get_years(all_tokens_phono)

for age in used_ages:
    for eval_phase in ['val', 'test']:
        for sample_func, sample_name in zip([sampling.sample_successes, sampling.sample_yyy], ['success', 'yyy']):

            print(f'for {sample_name}')

            phono_data_for_phase = all_tokens_phono[(all_tokens_phono.phase == phase) & (all_tokens_phono.partition == sample_name)]
            
            this_age_sample = sample_func(
                task_phase_to_sample_for= 'eval',
                test_split = 'Providence',
            test_dataset = 'all',
            raw_phono = phono_data_for_phase,
            age = age)       
            
            print('\tage sample', this_age_sample.shape)

for success
	age sample (87, 1)
for yyy
	age sample (105, 1)
for success
	age sample (87, 1)
for yyy
	age sample (105, 1)
for success
	age sample (500, 1)
for yyy
	age sample (500, 1)
for success
	age sample (500, 1)
for yyy
	age sample (500, 1)
for success
	age sample (500, 1)
for yyy
	age sample (500, 1)
for success
	age sample (500, 1)
for yyy
	age sample (500, 1)
for success
	age sample (500, 1)
for yyy
	age sample (500, 1)
for success
	age sample (500, 1)
for yyy
	age sample (500, 1)
for success
	age sample (500, 1)
for yyy
	age sample (500, 1)
for success
	age sample (500, 1)
for yyy
	age sample (500, 1)
for success
	age sample (500, 1)
for yyy
	age sample (500, 1)
for success
	age sample (500, 1)
for yyy
	age sample (500, 1)
for success
	age sample (500, 1)
for yyy
	age sample (118, 1)
for success
	age sample (500, 1)
for yyy
	age sample (118, 1)
for success
	age sample (0, 1)
for yyy
	age sample (0, 1)
for success
	age sample (0, 1)
for yyy
	age sample (0, 1)


In [7]:
# Write final all_tokens_phono with all split information to the proper place.
if not exists(config.prov_dir):
    os.makedirs(config.prov_dir)
    
all_tokens_phono.to_pickle(join(config.prov_dir, 'pvd_all_tokens_phono_for_eval.pkl'))

In [8]:
from datetime import datetime
print(datetime.today())

2022-11-19 19:38:29.374644
